In [1]:
from src.code import network
import pandas as pd
import os
from src.code import feature_extraction
from tensorflow import keras
import tensorflow as tf

### if you need to train your model, replace the filepath of the datasets.

In [2]:
def load_data():
    '''
    load datasets
    :return: datasets
    '''
    train_set = pd.read_csv('../../resources/datasets/dataset.csv')
    return train_set['seq'], train_set['label']

In [3]:
train_seq, train_label = load_data()
train_seq = train_seq.values.flatten().tolist()

In [4]:
fastText_model = feature_extraction.train_model(train_seq, 'model/fastText')

In [5]:
fastText_model.save('../../resources/model/fastText/fastText.model')

In [6]:
x_train = feature_extraction.get_features_from_datasets(train_seq, 'train', fastText_model, 'model/fastText')

In [7]:
afpn = network.create_model(x_train.shape)

In [8]:
file_path = os.path.join('../../resources/model', 'model.h5')
best_saving_1 = tf.keras.callbacks.ModelCheckpoint(file_path, monitor='val_final_output3_acc', mode='auto',
                                                           verbose=0, save_best_only=True, save_weights_only=True)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_final_output3_acc', patience=50)
afpn.fit(x_train, train_label, epochs=500, shuffle=True, validation_split=0.05, callbacks=[early_stopping, best_saving_1], verbose=0, batch_size=128)
afpn.load_weights(file_path)